In [1]:
!export CUDA_VISIBLE_DEVICES=5

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [3]:
import tensorflow as tf
from transformers import BertTokenizerFast, BertForSequenceClassification
import pandas as pd 
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import re

2024-07-12 21:44:26.262406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 21:44:26.285205: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 21:44:26.285232: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 21:44:26.299736: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 21:44:27.087979: W tensorflow/compiler/tf

In [4]:
if torch.cuda.is_available():
    print("CUDA is available. Number of GPUs:", torch.cuda.device_count())
    print("CUDA device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")
torch.cuda.set_device(torch.device("cuda:0"))
device = torch.device('cpu')

CUDA is available. Number of GPUs: 1
CUDA device name: NVIDIA GeForce GTX 1080 Ti


In [5]:
tokenizer = BertTokenizerFast.from_pretrained("google-bert/bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5).to(device)

/home/rpierson/anaconda3/envs/secondenvi/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]


2024-07-12 21:44:31.395969: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10532 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:85:00.0, compute capability: 6.1


In [7]:
strategy = tf.distribute.MirroredStrategy()
if torch.cuda.is_available():
        print(f"LOG: Torch allocated Memory: \
                {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB", flush=True)
        print(f"LOG: Torch cached Memory: \
                {torch.cuda.memory_reserved() / 1024 ** 2:.2f} MB", flush=True)

if tf.config.experimental.list_physical_devices('GPU'):
    zero_info = tf.config.experimental.get_memory_info('GPU:0')
    print(f"LOG: TensorFlow, 0: Current memory usage: \
            {zero_info['current'] / 1024 ** 2:.2f} MB", flush=True)
    print(f"LOG: TensorFlow, 0: Peak memory usage: \
            {zero_info['peak'] / 1024 ** 2:.2f} MB", flush=True)

# model.compile(optimizer=Adam(3e-5))  # No loss argument!

#model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
LOG: Torch allocated Memory:                 0.00 MB
LOG: Torch cached Memory:                 0.00 MB
LOG: TensorFlow, 0: Current memory usage:             0.00 MB
LOG: TensorFlow, 0: Peak memory usage:             0.00 MB


In [8]:
#Now working with the text

In [9]:
df = pd.read_csv(f'/home/rpierson/Topic_Files/topic_0.csv')
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
max_length = 512
def tokenize_function(text):
    return tokenizer(text, padding= "max_length", truncation=True)


In [11]:
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [12]:
# Apply the tokenize function to the 'Combined_Text' column
#for x in range(len(train_df)):
#    train_df.loc[x, 'Combined_Text'] = tokenize_function(train_df.loc[x, 'Combined_Text'])


#train_df['Combined_Text'] = train_df['Combined_Text'].apply(tokenize_function)
#val_df['Combined_Text'] = val_df['Combined_Text'].apply(tokenize_function)

In [13]:
train_encodings = tokenizer(
    train_df['Combined_Text'].tolist(), 
    truncation=True, 
    padding=True, 
    max_length=512
)

val_encodings = tokenizer(
    val_df['Combined_Text'].tolist(), 
    truncation=True, 
    padding=True, 
    max_length=512
)


In [14]:
train_inputs = {
    'input_ids': tf.constant(train_encodings['input_ids']),
    'attention_mask': tf.constant(train_encodings['attention_mask'])
}

val_inputs = {
    'input_ids': tf.constant(val_encodings['input_ids']),
    'attention_mask': tf.constant(val_encodings['attention_mask'])
}


In [15]:
train_labels = tf.constant(train_df['Priority'].values)  # Shape (num_samples,)
val_labels = tf.constant(val_df['Priority'].values) 

<tf.Tensor: shape=(824,), dtype=int64, numpy=
array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 1, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 1,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 1, 3, 3, 3, 1, 2, 3, 3, 3, 4, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 2, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 2, 1, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3,
       3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
     

In [16]:
print("Train input_ids shape:", tf.constant(train_encodings['input_ids']).shape)
print("Train attention_mask shape:", tf.constant(train_encodings['attention_mask']).shape)
print("Val input_ids shape:", tf.constant(val_encodings['input_ids']).shape)
print("Val attention_mask shape:", tf.constant(val_encodings['attention_mask']).shape)

Train input_ids shape: (824, 512)
Train attention_mask shape: (824, 512)
Val input_ids shape: (206, 512)
Val attention_mask shape: (206, 512)


In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_encodings, train_labels)).shuffle(len(train_df)).batch(15)
val_dataset = tf.data.Dataset.from_tensor_slices((val_encodings, val_labels)).batch(15)


In [18]:
# Create TensorFlow datasets
#train_dataset = tf.data.Dataset.from_tensor_slices((
#    dict(train_encodings),
#    train_df['Priority']
#)).shuffle(len(train_df)).batch(15)

#val_dataset = tf.data.Dataset.from_tensor_slices((
#    dict(val_encodings),
#    val_df['Priority']
#)).batch(15)

In [19]:
for inputs, labels in train_dataset.take(1):
    print("Train batch input_ids shape:", inputs['input_ids'].shape)
    print("Train batch attention_mask shape:", inputs['attention_mask'].shape)
    print("Train batch labels shape:", labels.shape)

for inputs, labels in val_dataset.take(1):
    print("Val batch input_ids shape:", inputs['input_ids'].shape)
    print("Val batch attention_mask shape:", inputs['attention_mask'].shape)
    print("Val batch labels shape:", labels.shape)

Train batch input_ids shape: (15, 512)
Train batch attention_mask shape: (15, 512)
Train batch labels shape: (15,)
Val batch input_ids shape: (15, 512)
Val batch attention_mask shape: (15, 512)
Val batch labels shape: (15,)


2024-07-12 21:44:34.511253: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-12 21:44:34.521590: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [20]:
!export TF_GPU_ALLOCATOR=cuda_malloc_async

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
for step, batch in enumerate(train_dataset):
    print(f"Step {step}: Batch structure = {type(batch)}")
    print(batch)
    if step >= 1:
        break

Step 0: Batch structure = <class 'tuple'>
({'input_ids': <tf.Tensor: shape=(15, 512), dtype=int32, numpy=
array([[  101,  1031,  2393, ...,     0,     0,     0],
       [  101,  3945, 12117, ...,     0,     0,     0],
       [  101,  9986,  8917, ...,     0,     0,     0],
       ...,
       [  101,  2770, 13232, ...,     0,     0,     0],
       [  101,  1031,  3145, ...,     0,     0,     0],
       [  101,  4773, 16602, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(15, 512), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(15, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
    

In [37]:
from tqdm import tqdm

In [42]:

num_epochs = 15
loss_fn = torch.nn.CrossEntropyLoss()

# Set model to training mode
model.train()

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for step, batch in enumerate(train_dataset):
        # Extracting the input tensors and labels
        inputs = torch.tensor(batch[0]['input_ids'].numpy()).squeeze().to(device)
        token_type_ids = torch.tensor(batch[0]['token_type_ids'].numpy()).squeeze().to(device)
        attention_mask = torch.tensor(batch[0]['attention_mask'].numpy()).squeeze().to(device)
        labels = torch.tensor(batch[1].numpy()-1).squeeze().to(device)
        
        # Forward pass
        outputs = model(input_ids=inputs, token_type_ids=token_type_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        # Compute loss
        loss = loss_fn(logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Optional: Calculate and print metrics, etc.
        total_loss += loss.item()
        if (step + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{step + 1}/{len(train_dataset)}], Loss: {total_loss / 100:.4f}")
            total_loss = 0.0

    # Optional: Perform evaluation or save checkpoints at the end of each epoch
    # Example:
    # evaluate_model()
    # save_checkpoint()

print("Training finished!")

AttributeError: 'Adam' object has no attribute 'zero_grad'

In [ ]:
from torch.optim import Adam
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast

optimizer = Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

scaler = GradScaler()  
# Assuming train_dataset is your DataLoader or Dataset
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for step, batch in enumerate(train_dataset):
        inputs = torch.tensor(batch[0]['input_ids'].numpy()).squeeze().to(device)
        token_type_ids = torch.tensor(batch[0]['token_type_ids'].numpy()).squeeze().to(device)
        attention_mask = torch.tensor(batch[0]['attention_mask'].numpy()).squeeze().to(device)
        labels = torch.tensor(batch[1].numpy() - 1).to(device)  # Adjust labels to start from 0

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(input_ids=inputs, token_type_ids=token_type_ids, attention_mask=attention_mask)
            logits = outputs.logits
            logits = logits.reshape(-1, num_classes)

            loss = loss_fn(logits, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        if (step + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{step + 1}/{len(train_dataset)}], Loss: {total_loss / 100:.4f}")
            total_loss = 0.0

print("Training finished!")

2024-07-12 22:11:31.044229: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-12 22:19:09.116185: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-12 22:26:30.668621: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-12 22:32:02.561043: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-12 22:37:44.161837: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
